In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
from llama_index import SimpleDirectoryReader
from llama_index.node_parser import SimpleNodeParser
from llama_index.llms import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index import download_loader
from llama_index import ServiceContext
from llama_index import VectorStoreIndex
from llama_index import StorageContext
from llama_index.vector_stores import PineconeVectorStore
import pinecone
from tqdm.autonotebook import tqdm
from pinecone import Pinecone
from pinecone import ServerlessSpec

c:\anaconda3\envs\deeplearning\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
pinecone_client = Pinecone(
    api_key=os.environ["PINECONE_API_KEY"], 
    environment=os.environ["PINECONE_ENVIRONMENT"]
)

In [4]:
pinecone_client

In [5]:
UnstructuredReader= download_loader("UnstructuredReader")

In [6]:
UnstructuredReader

custom_module.UnstructuredReader

In [7]:
dir_reader = SimpleDirectoryReader(
    input_dir=r"C:\github_repos\LlamaIndex-Develop-LLM-Powered-Applications-With-LlamaIndex\data\llamaindex-docs-tmp",
    file_extractor={".html": UnstructuredReader()},
)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\roman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\roman\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
dir_reader

In [9]:
documents= dir_reader.load_data()

In [10]:
documents

[Document(id_='eeee43c6-c116-4246-a77b-6a66ed6f2758', embedding=None, metadata={'file_path': 'C:\\github_repos\\LlamaIndex-Develop-LLM-Powered-Applications-With-LlamaIndex\\data\\llamaindex-docs-tmp\\agents.html', 'file_name': 'agents.html', 'file_type': 'text/html', 'file_size': 154884, 'creation_date': '2023-10-18', 'last_modified_date': '2024-02-04', 'last_accessed_date': '2024-02-04', 'filename': 'C:\\github_repos\\LlamaIndex-Develop-LLM-Powered-Applications-With-LlamaIndex\\data\\llamaindex-docs-tmp\\agents.html'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="Hide navigation sidebar\n\nHide table of contents sidebar\n\nToggle site navigation sidebar\n\nLlamaIndex 🦙 0.8.45.post1\n\nToggle Light / Dark / Auto color theme\n\nToggle table of contents

In [11]:
print(documents)

[Document(id_='eeee43c6-c116-4246-a77b-6a66ed6f2758', embedding=None, metadata={'file_path': 'C:\\github_repos\\LlamaIndex-Develop-LLM-Powered-Applications-With-LlamaIndex\\data\\llamaindex-docs-tmp\\agents.html', 'file_name': 'agents.html', 'file_type': 'text/html', 'file_size': 154884, 'creation_date': '2023-10-18', 'last_modified_date': '2024-02-04', 'last_accessed_date': '2024-02-04', 'filename': 'C:\\github_repos\\LlamaIndex-Develop-LLM-Powered-Applications-With-LlamaIndex\\data\\llamaindex-docs-tmp\\agents.html'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="Hide navigation sidebar\n\nHide table of contents sidebar\n\nToggle site navigation sidebar\n\nLlamaIndex 🦙 0.8.45.post1\n\nToggle Light / Dark / Auto color theme\n\nToggle table of contents

In [12]:
documents = dir_reader.load_data()
node_parser = SimpleNodeParser.from_defaults(chunk_size=500, chunk_overlap=20)

In [13]:
documents

[Document(id_='9ff0d8ca-b072-41d6-bf01-4e9352c0ae38', embedding=None, metadata={'file_path': 'C:\\github_repos\\LlamaIndex-Develop-LLM-Powered-Applications-With-LlamaIndex\\data\\llamaindex-docs-tmp\\agents.html', 'file_name': 'agents.html', 'file_type': 'text/html', 'file_size': 154884, 'creation_date': '2023-10-18', 'last_modified_date': '2024-02-04', 'last_accessed_date': '2024-02-04', 'filename': 'C:\\github_repos\\LlamaIndex-Develop-LLM-Powered-Applications-With-LlamaIndex\\data\\llamaindex-docs-tmp\\agents.html'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="Hide navigation sidebar\n\nHide table of contents sidebar\n\nToggle site navigation sidebar\n\nLlamaIndex 🦙 0.8.45.post1\n\nToggle Light / Dark / Auto color theme\n\nToggle table of contents

In [14]:
node_parser

SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x000001BE01C48460>, id_func=<function default_id_func at 0x000001BE016DDA20>, chunk_size=500, chunk_overlap=20, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')

In [15]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
embed_model = OpenAIEmbedding(model="text-embedding-ada-002", embed_batch_size=100)
service_context = ServiceContext.from_defaults(
llm=llm, embed_model=embed_model, node_parser=node_parser)

In [16]:
index_name = "llamaindex-documentation-helper"

In [17]:
pinecone_index = pinecone_client.Index(index_name=index_name,host=os.environ["PINECONE_HOST"])

In [18]:
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [19]:
index = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=storage_context,
    service_context=service_context,
    show_progress=True,
    show_progress_bar=True)
print("Finished Ingesting...")

Upserted vectors: 100%|██████████| 14/14 [00:00<00:00, 20.29it/s]

Finished Ingesting...
